Working with Neural Network Models

© Hans Nieminen, Satakunta University of Applied Sciences

# Exercise 7.1

For this exercise use the GPU.

In [ ]:
import numpy as np
import torch

In [ ]:
torch.manual_seed(222)
t_cpu = torch.randint(-1000, 1001, (100, 1000, 1000))

In [ ]:
if torch.cuda.is_available():
    # Move tensor to GPU
    t_gpu = t_cpu.to('cuda')

In [ ]:
if torch.cuda.is_available():
    # Compute the mean of all elements in the tensor on the GPU
    tensor_mean_gpu = t_gpu.float().mean()
    # Compute the standard deviation of all elements in the tensor on the GPU
    tensor_std_gpu = t_gpu.float().std()

In [ ]:
if torch.cuda.is_available():
    # Move the results back to the CPU
    tensor_mean_cpu = tensor_mean_gpu.to('cpu')
    tensor_std_cpu = tensor_std_gpu.to('cpu')

In [ ]:
tensor_mean_cpu

tensor(0.0029)

In [ ]:
tensor_std_cpu

tensor(577.6354)

In [ ]:
t1_cpu = torch.cat((tensor_mean_cpu.reshape(1),
                    tensor_std_cpu.reshape(1)),
                   dim=0)
t1_cpu

tensor([2.8614e-03, 5.7764e+02])

In [ ]:
torch.manual_seed(888)
t2_cpu = torch.randint(1, 11, (10,2))

In [ ]:
t2_cpu

tensor([[ 3,  1],
        [10,  6],
        [ 1,  6],
        [ 2,  3],
        [ 8,  5],
        [10,  4],
        [ 9,  8],
        [ 1,  5],
        [10, 10],
        [ 1,  3]])

In [ ]:
product = torch.matmul(t2_cpu.float(), t1_cpu)

In [ ]:
product

tensor([ 577.6440, 3465.8413, 3465.8154, 1732.9120, 2888.2002, 2310.5703,
        4621.1094, 2888.1799, 5776.3833, 1732.9091])

In [ ]:
print(product.numpy().mean().round(2))

2945.96


# Exercise 7.2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class Dataset0702(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        input = self.X[idx]
        label = self.y[idx]
        if self.transform is not None:
            for trf in self.transform:
                input = trf(input)
        input = torch.tensor(input, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.float32)
        return input, label

In [ ]:
X, y = load_iris(return_X_y=True)

In [ ]:
X[:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [ ]:
X.shape

(150, 4)

In [ ]:
y[:3]

array([0, 0, 0])

In [ ]:
def Squared(x):
    return x**2

def Log(x):
    return np.log(x)

In [ ]:
Log(Squared(X))[:3]

array([[ 3.25848108,  2.50552594,  0.67294447, -3.21887582],
       [ 3.17847041,  2.19722458,  0.67294447, -3.21887582],
       [ 3.09512502,  2.32630162,  0.52472853, -3.21887582]])

In [ ]:
transform = [Squared, Log]

In [ ]:
transformed_dataset = Dataset0702(X,
                                  y,
                                  transform=transform)

In [ ]:
transformed_dataloader = DataLoader(transformed_dataset,
                                    batch_size=16,
                                    shuffle=False)

In [ ]:
for batch_idx, (inputs, labels) in enumerate(transformed_dataloader):
    if batch_idx == 7:
        print(f"Transformed Batch {batch_idx + 1}")
        batch8_inputs = inputs
        batch8_labels = labels

Transformed Batch 8


In [ ]:
batch8_inputs.shape

torch.Size([16, 4])

In [ ]:
batch8_inputs

tensor([[3.8338, 2.1972, 3.4095, 1.4839],
        [3.4809, 1.8326, 3.2189, 1.3863],
        [3.5157, 2.0592, 3.2585, 1.7509],
        [3.7126, 2.3263, 3.3354, 1.6658],
        [3.7436, 2.1972, 3.4095, 1.1756],
        [4.0824, 2.6700, 3.8042, 1.5769],
        [4.0824, 1.9110, 3.8630, 1.6658],
        [3.5835, 1.5769, 3.2189, 0.8109],
        [3.8630, 2.3263, 3.4809, 1.6658],
        [3.4455, 2.0592, 3.1785, 1.3863],
        [4.0824, 2.0592, 3.8042, 1.3863],
        [3.6811, 1.9865, 3.1785, 1.1756],
        [3.8042, 2.3878, 3.4809, 1.4839],
        [3.9482, 2.3263, 3.5835, 1.1756],
        [3.6491, 2.0592, 3.1372, 1.1756],
        [3.6166, 2.1972, 3.1785, 1.1756]])

In [ ]:
batch8_labels

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [ ]:
print(batch8_inputs.numpy()[:,1].mean().round(3))

2.136


# Exercise 7.3

In [ ]:
import numpy as np
import torch

In [ ]:
vector = [3.2, 2.1, 0.6, 8.3]
x = torch.tensor(vector, requires_grad=True)

In [ ]:
# Perform a series of operations on this tensor
y = x * 2
z = y * y * 3
q = z.sum()
out = q.sqrt()

In [ ]:
out

tensor(31.7301, grad_fn=<SqrtBackward0>)

In [ ]:
print("Tensor x:", x)
print("Tensor y (x * 2):", y)
print("Tensor z (y * y * 3):", z)
print("Tensor q (sum of z):", z)
print("Output (squared root of q):", out)

Tensor x: tensor([3.2000, 2.1000, 0.6000, 8.3000], requires_grad=True)
Tensor y (x * 2): tensor([ 6.4000,  4.2000,  1.2000, 16.6000], grad_fn=<MulBackward0>)
Tensor z (y * y * 3): tensor([122.8800,  52.9200,   4.3200, 826.6800], grad_fn=<MulBackward0>)
Tensor q (sum of z): tensor([122.8800,  52.9200,   4.3200, 826.6800], grad_fn=<MulBackward0>)
Output (squared root of q): tensor(31.7301, grad_fn=<SqrtBackward0>)


In [ ]:
# Compute the gradient of out with respect to x
out.backward()

In [ ]:
# Access and print the gradients
print("Gradients of x:", x.grad)
print("The sum of gradients of x:", x.grad.sum().numpy().round(2))

Gradients of x: tensor([1.2102, 0.7942, 0.2269, 3.1390])
The sum of gradients of x: 5.37
